# Post process for ensembling average
    - layer:8, neuron:72
- Define task to postprocess
-  task  = ['predict', 'cross_validation']
    - cross_validation is for post process
- 'cross_validation': for cross validation; has 7 folds
- 'predict': for test using parameters chose by cross validation; has 1 fold --> fold8
-  force = ['CL', 'CD']
-  filt  = ['bessel', 'no filter']
    - no filter for all CL, CD

# import packages

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.signal import bessel
from scipy.signal import filtfilt

# define the font style in the figures
- title, label, legend, text, math text
- font: Times New Roman
- font size: 20

In [ ]:
import matplotlib as mpl

mpl.rcParams['font.family'] = 'Times New Roman'
mpl.rcParams['font.size']   = 20 # 15

mpl.rcParams['mathtext.fontset'] = 'custom'
mpl.rcParams['mathtext.rm'] = 'Times New Roman'
mpl.rcParams['mathtext.it'] = 'Times New Roman:italic'

# plot half delta wing in 2D

In [ ]:
x = np.linspace(-1, 1, 100)
y = x+1
fig,ax = plt.subplots(figsize=(5,3))
ax.plot(x, y)
ax.set_xlim(-1, 0)
ax.set_ylim(0, 1)
ax.set_aspect('equal')
ax.set_xlabel('$x/c$')
ax.set_ylabel('$y/c$')
ax.grid(color='k', linestyle='-', linewidth=0.1)
fig.savefig('deltawing.svg')

# define task

In [ ]:
task = 'predict' 
force = 'CL'     
filt = 'bessel' # only for cp; no cl or cd filter

if task == 'cross_validation':
    folds = 7
    print('Post process for ', force)
elif task == 'predict':
    folds = 1
else:
    print('Error. Please define your task.') 
print('Define the task complete.')

# load data

## define the hyperparameters used in trained model

In [ ]:
#Load the variable from ensemble average prediction; only (layer,neuron): (3,32), (5,64), (8,72) for ensembling
num_neuron = [72] #[32,64,72]      [16, 24, 32, 40, 48, 56, 64, 72]
num_layer  = [8]  #[3,5,8]         [1,  2,  3  ,4,  5,  6,  7,  8]
epoch_total = 200
weight_decay = 0 # 5e-4 # l2 regularization hyperparameter
learn_rate   = 1e-5 # 1e-4
batch_how    = 200  # 200

ensemble_num = 30

## load data

In [ ]:
# read data
# folder path
data_path = 'gust_dataframe.csv'

# read the excel data
gust_origin = pd.read_csv(data_path)

In [ ]:
# check data
gust_origin.head()

In [ ]:
# set the first column as index, i.e. set case_nn as index, for convenience
gust_origin.set_index('case_number', inplace=True)
print(gust_origin.shape)
# check the first 5 columns
gust_origin.head()

# filter data

In [ ]:
# bessel filter for every column; only filt CP
if filt == 'bessel':
    
    gust_index   = gust_origin.index
    gust_column  = gust_origin.columns
    gust_no_filter = np.zeros((gust_index.nunique(),len(gust_origin.loc[['case_01']]),len(gust_origin.columns))) # (32,2500,18)
    gust_filter    = np.zeros((gust_index.nunique(),len(gust_origin.loc[['case_01']]),len(gust_origin.columns)))
    case_all_no = (np.linspace(0,31,32)).astype(int)
    b, a = bessel(8, 0.01) # define the bessel filter; 8 is the filter order; 0.01 is the W_n (between 0 to 1)
    
    for i in case_all_no:      # i is the case number
        case_numer = 'case_' + str(i+1).zfill(2)
        gust_no_filter[i] = gust_origin.loc[[case_numer]].to_numpy()
        for j in range(len(gust_origin.columns)-2):       # j is the CP and CL CD number
            gust_filter[i][:,j] = filtfilt(b, a, gust_no_filter[i][:,j]) # bessel filter for cp
        gust_filter[i][:,len(gust_origin.columns)-2] = gust_no_filter[i][:,len(gust_origin.columns)-2]  # no filter for CL
        gust_filter[i][:,len(gust_origin.columns)-1] = gust_no_filter[i][:,len(gust_origin.columns)-1]  # no filter for CD
    gust = pd.DataFrame(data=np.concatenate(gust_filter,axis=0), index=gust_index, columns=gust_column)
    
elif filt == 'no filter':
    gust = gust_origin
    
else:
    print('Error. Please define your filter.')

In [ ]:
# check gust
gust.head()

In [ ]:
# check force
plt.plot(gust[[force]].to_numpy()[0:2500,0])

# retrieve cp cl and cd from dataframe

In [ ]:
# retrieve CP, 16(CP) + 16(CP_1st_order) + 16(CP_2nd_order) = 48 columns in total
# simply drop CL and CD, then the rest is CP
CP = gust.drop(['CL','CD'],axis=1, inplace=False)
CP.head()

In [ ]:
# retrieve CL or CD
if force == 'CL':
    CL = gust[['CL']]
elif force == 'CD':
    CD = gust[['CD']]
else:
    print('Error. Please define your force to post process.')
gust[['CL']].head()

In [ ]:
# retrieve for 8 different folds
# read the split .txt file; the number means the cases for test
split_path = 'delta_wing_32cases_split_random.txt'
with open(split_path, 'r') as file1:    
    fold_split = file1.read().splitlines()       # split by \n
fold_split = fold_split[1:] # drop the head line
for i in range(8):
    fold_split[i] = fold_split[i][9:].split()    # split by space; the number starts from 10th character.
    for j in range(4):
        fold_split[i][j] = int(fold_split[i][j]) # convert string to int
fold_split

In [ ]:
# define the 32 case number
case_all_num = np.arange(gust.index.nunique()) + 1
print(case_all_num)
print(case_all_num.shape)

In [ ]:
# find the case number for train and test cases;
if task == 'cross_validation':
    train_case = [ [0]*24 for _ in range(folds) ] # initialize 2d list; list for 'case_nn'
    test_case  = [ [0]*4 for _ in range(folds) ]
    # find case number for cross validation folds: fold1~fold7
    for i in range(folds): 
        train_case_temp = np.setdiff1d( case_all_num, np.concatenate((fold_split[i], fold_split[-1]))) 
        # drop test case and the last one (which is for fold 8 , final test)
        test_case_temp  = fold_split[i]

        # conbine to 'case_nn' sample for train and test case; prepare for next step
        for j in range(len(train_case_temp)):
            train_case[i][j] = 'case_' + str(train_case_temp[j]).zfill(2)
        for k in range(len(test_case_temp)):
            test_case[i][k]  = 'case_' + str(test_case_temp[k]).zfill(2)
            
elif task == 'predict': 
    train_case = [ [0]*28 for _ in range(folds) ] # initialize 2d list; list for 'case_nn'
    test_case  = [ [0]*4 for _ in range(folds) ]
    # find case number for test fold: fold8  
    train_case_temp = np.setdiff1d(case_all_num, fold_split[-1])
    test_case_temp  = fold_split[-1]
    for jj in range(len(train_case_temp)):
        train_case[-1][jj] = 'case_' + str(train_case_temp[jj]).zfill(2)
    for kk in range(len(test_case_temp)):
        test_case[-1][kk]  = 'case_' + str(test_case_temp[kk]).zfill(2)

else:
    print('Error. Please define your task.')

print(test_case[0])
print('Done!')

#  prepare for train and test data sets

In [ ]:
if task == 'cross_validation':
    F_train_rec = np.zeros([folds,60000,1]) # 24*2500 = 60000
    F_test_rec  = np.zeros([folds,10000,1])  # 4*2500  = 10000
    P_train_rec = np.zeros([folds,60000,48])
    P_test_rec  = np.zeros([folds,10000,48])

    for i in range(folds): # 7 folds for training, the last one is for test, not retrieve here, since this is for training
        if force == 'CL':
            F_train_rec[i] = CL.loc[train_case[i]].to_numpy()
            F_test_rec[i]  = CL.loc[test_case[i]].to_numpy()
        elif force == 'CD':
            F_train_rec[i] = CD.loc[train_case[i]].to_numpy()
            F_test_rec[i]  = CD.loc[test_case[i]].to_numpy()
        else:
            print('Error. Please define force.')

        P_train_rec[i] = CP.loc[train_case[i]].to_numpy()
        P_test_rec[i]  = CP.loc[test_case[i]].to_numpy()

elif task == 'predict':
    F_train_rec = np.zeros([folds,70000,1]) # 28*2500 = 70000
    F_test_rec  = np.zeros([folds,10000,1])  # 4*2500  = 10000
    P_train_rec = np.zeros([folds,70000,16])
    P_test_rec  = np.zeros([folds,10000,16])
    
    if force == 'CL':
        F_train_rec[0] = CL.loc[train_case[0]].to_numpy()
        F_test_rec[0]  = CL.loc[test_case[0]].to_numpy()
    elif force == 'CD':
        F_train_rec[0] = CD.loc[train_case[0]].to_numpy()
        F_test_rec[0]  = CD.loc[test_case[0]].to_numpy()
    else:
        print('Error. Please define force.')

    P_train_rec[0] = CP.loc[train_case[0]].to_numpy()
    P_test_rec[0]  = CP.loc[test_case[0]].to_numpy()
    
else:
    print('Error. Please define your task.')
    
print('Done')

In [ ]:
print(task)
print(force)
print(folds)

# load the ensembling average results

In [ ]:
result_folder = 'deltaWing/code/pred-cl-ensemble-average-cp-besselfilter/'
predictions = {}
for i in num_neuron: # define the neurons
    for j in num_layer: # define the hidden layers 
        for k in range(ensemble_num): # start loop for 7 folds cross-validation
            # print('loading ensemble_num: %d' % (i))
            predictions[k,i,j] = np.array(pickle.load(
                open(result_folder+'F_pred_p_filter_ensemble_num_'+str(k)+'_neuron_'+str(i)+'_layer_'+str(j)+'.dat', 'rb')))
            # respahe predictions (200,10000,1) --> (200,10000)
            predictions[k,i,j] = predictions[k,i,j].reshape(epoch_total,len(P_test_rec[0])).T

print(predictions[0,72,8].shape)
print('Done!')
            

# define ensemble_average func
- finds the ensemble average of the epoch predictions and the std  history
- it returns the average result (array) and the std (array)

In [ ]:
def ensemble_average(epoch_pred):
    sum_temp = np.zeros((len(epoch_pred[:,0]),len(epoch_pred[0,:])))
    sum_temp[:,0] = epoch_pred[:,0]
    std_hist = np.zeros((len(epoch_pred[0,:])))
    std_hist[0] = np.std(sum_temp[:,0])
    ensemble_averg = np.zeros((len(epoch_pred[:,0]),len(epoch_pred[0,:])))
    ensemble_averg[:,0] = epoch_pred[:,0]
    for i in range(len(epoch_pred[0,:])-1):
        sum_temp[:,i+1] = sum_temp[:,i] + epoch_pred[:,i+1]
        std_hist[i+1] = np.std(sum_temp[:,i+1]/(i+2))
        ensemble_averg[:,i+1] = sum_temp[:,i+1]/(i+2)
    return(std_hist, ensemble_averg)

In [ ]:
# test function ensemble_average
a = np.array([[1,1,2,2],[1,2,1,3],[1,1,1,5]])
(b,c) = ensemble_average(a)
print(c)
print(b == np.std(np.array([[1,2/2,4/3,6/4],[1,3/2,4/3,7/4],[1,2/2,3/3,8/4]]),axis=0))

In [ ]:
# this cell calcalute the mse of different ensembling average: 50,60,70,80,90,...200
from sklearn.metrics import mean_squared_error as mse

ensemble_use = np.arange(50,210,10)
mse_ensemble_use  = {} # dictionary, with keys and values
epoch_hist_use    = np.zeros([len(ensemble_use),len(F_test_rec[0]),ensemble_num]) # 16*10000*30: 16 to show the mse, 10000 samples, 30 repeats
ensemble_avrg_use = {}
ensemble_std_use  = {}
ensemble_avrg_use = {}

neuron_this = num_neuron[0]
layer_this  = num_layer[0]

fig,ax_use = plt.subplots(figsize=(5,3))
for j in ensemble_use: # [50,60,70,80,90,...200]
    for i in range(ensemble_num): # repeat ensemble_num times with same hyperparameters
        epoch_hist_use[int(j/10-5),:,i] = predictions[i,neuron_this,layer_this][:,j-1]
        
    (ensemble_std_use[j], ensemble_avrg_use[j]) = ensemble_average(epoch_hist_use[int(j/10-5)])
    
    mse_ensemble_use[j] = mse(ensemble_avrg_use[j][:,-1], F_test_rec[0][:,:])

ax_use.plot(mse_ensemble_use.keys(),mse_ensemble_use.values(),color='blue', linewidth=1.5,marker='.',markersize=8)
ax_use.set_xlim(50,200)
ax_use.set_xticks(np.arange(50,210,30).tolist())
# ax_use.set_ylim(0.012,0.030)
ax_use.grid(color='k', linestyle='-', linewidth=0.1)
ax_use.set_xlabel('Epochs')
ax_use.set_ylabel('MSE')
# fig.savefig('ensemble_error_Filter_neuron%d_layer%d.png' % (neuron_this,layer_this), bbox_inches='tight', dpi=300)
# fig.savefig('ensemble_error_Filter_neuron%d_layer%d.svg' % (neuron_this,layer_this), bbox_inches='tight

In [ ]:
fig, ax = plt.subplots(figsize=(5,3))
ax.plot(ensemble_std_use[200],color='blue', linewidth=1.5,marker='.',markersize=8)
ax.grid(color='k', linestyle='-', linewidth=0.1)
ax.set_xlabel('Number of Repeats')
ax.set_ylabel('STD')
ax.margins(x=0)
ax.set_xlim(0,30)
# fig.savefig('STD_repeat_Filter_neuron%d_layer%d.png' % (neuron_this,layer_this), bbox_inches='tight', dpi=300)
# fig.savefig('STD_repeat_Filter_neuron%d_layer%d.svg' % (neuron_this,layer_this), bbox_inches='tight')

In [ ]:
fig, ax_split = plt.subplots(2,2,figsize=(8,4))
repeat_num = 200 # repeated number: 50 100

T = 0.3 # period is 0.3s
f = 1000 # frequency is 1000Hz
t_star = (np.arange(0,10000) - 500) / f / T 

fig_num = 0 # for count opnly
for i in range(4):
    if fig_num in [0,1]:
        p = 0
        q = fig_num
    else:
        p = 1
        q = fig_num - 2
    ax_split[p,q].plot(t_star[0:2500-1], F_test_rec[0][2500*i:2500*(i+1)-1,:],label='F_test',color='red', linewidth=2)
    ax_split[p,q].plot(t_star[0:2500-1], ensemble_avrg_use[repeat_num][2500*i:2500*(i+1)-1,-1],label='pred_average', 
                       color='blue', linewidth=2, linestyle='--') # 50 and 200 epochs
    ax_split[p,q].grid(color='k', linestyle='-', linewidth=0.1)
    ax_split[p,q].set_xticks(np.arange(-1,8))
    ax_split[p,q].set_xlim(-1,6.5)
    ax_split[p,q].margins(x=0)
    ax_split[p,q].set_xlabel('$t^*$')
    fig_num = fig_num + 1
ax_split[0,0].set_ylabel('$C_{L}$')
ax_split[0,0].legend(['$C_{L}$, measured','$C_{L}$, predicted'])
ax_split[1,0].set_ylabel('$C_{L}$')

# fig.savefig('ensemble%d_pred_besselFilter_neuron%d_layer%d.png' % (repeat_num,neuron_this,layer_this), bbox_inches='tight', dpi=300)
# fig.savefig('ensemble%d_pred_besselFilter_neuron%d_layer%d.svg' % (repeat_num,neuron_this,layer_this), bbox_inches='tight')

# save results to combine the data with bessel filter

In [ ]:
import pickle

# #Save the variable
# pickle.dump(predictions, open("predictions.dat", "wb"))
# #Load the variable
# variable = pickle.load(open("predictions.dat", "rb"))

pickle.dump(mse_ensemble_use, open("../2023-01-16-mse-std-pred/mse_ensemble_use_bessel_filter.dat", "wb"))
pickle.dump(ensemble_std_use, open("../2023-01-16-mse-std-pred/ensemble_std_use_bessel_filter.dat", "wb"))
# pickle.dump(F_test_rec, open("../2023-01-16-mse-std-pred/F_test_rec.dat", "wb"))
pickle.dump(ensemble_avrg_use, open("../2023-01-16-mse-std-pred/ensemble_avrg_use_bessel_filter.dat", "wb"))